In [26]:
import numpy as np
from matplotlib import image, pyplot as plt
from math import floor, ceil

def backsub(U,b):
  """
  backsub(U,b)
  Solve the upper-triangular linear system with matrix U and right-hand side
  vector b.
  """
  n = len(b)
  x = np.zeros(n)
  for i in range(n-1,-1,-1):
    s = U[i,i+1:] @ x[i+1:]
    x[i] = ( b[i] - s ) / U[i,i]
  return x

def lsqrfact(A,b):
  """
  lsqrfact(A,b)
  Solve a linear least squares problem by QR factorization. Returns the
  minimizer of ||b-Ax||.
  """

  Q,R = np.linalg.qr(A, mode = 'reduced')
  c = Q.T@b
  x = backsub(R,c)

  return x


def loadface(imagedir,subject,pose):
  """
  Load in the face for the given subject number (integer) and the given
  pose (integer). Directory of images is passed in as imagedir.
  """
  filename =  f"{imagedir}/s{subject}/{pose}.pgm"
  image_vectors = image.imread(filename).astype(np.double)  # read, convert to double precision
  return image_vectors.flatten()

def showfaces(image_vectors):
  """
  Accepts a matrix of image vectors (assumed to be from 112 x 92 images, and
  with the image vectors as columns) and plots them in a grid.
  Will plot at most 16 faces.
  """
  n = image_vectors.shape[1]
  if n > 16:
    raise ValueError('A maximum of 16 faces please!')
  rows = ceil(n/4)
  cols = 4

  fig = plt.figure(figsize=(10,10))
  for j in range(n):
    pic = image_vectors[:,j].reshape(112,92)
    ax = fig.add_subplot(rows,cols,j+1)
    ax.axis('off')
    ax.imshow(pic, cmap='gray')

In [38]:
imagedir = 'attfaces-python'
faces = np.zeros([10304, 150])
face_id = np.zeros([150])
counter = 0
# B = np.zeros([10304,150])
for subj in range(25):
  face_id[counter] = subj
  for pose in range(6):
    faces[:,counter] = loadface(imagedir,subj+1,pose+1)
    counter += 1

# print(loadface('attfaces-python', 1, 1).reshape(10304, 1))
# faces[:,1].shape
# showfaces(faces[:,:1])
Q_faces, R_faces = np.linalg.qr(faces, mode = 'reduced')


In [42]:
def identiface(Q, R, subj, z):
  # similarity = np.zeros([150])
  qtz = Q.T@z
  similarity = backsub(R, qtz)
  confidence = np.zeros([25])
  for subject in range(25):
    confidence[subject] = sum(similarity[(subject * 6):(subject * 6) + 5])
  print(confidence)
  print(similarity)

  print("Bang!")

identiface(Q_faces, R_faces, face_id, faces[:,1:2])

[ 1.00000000e+00  6.61676853e-16 -4.10445451e-16 -5.38711393e-16
  6.44824455e-16  8.18367377e-16 -2.07933630e-15 -1.72947086e-16
 -2.20312163e-16 -6.76130484e-16  1.20324134e-16  1.20871305e-15
 -1.07115947e-15 -2.55409679e-16  4.62520114e-17  3.24628753e-16
 -1.32782926e-16  5.76928131e-17  3.53006141e-16  7.68457210e-16
 -2.24258861e-16 -2.99837577e-16 -2.98434956e-16 -8.74338845e-17
 -2.04957716e-16]
[-2.19205184e-15  1.00000000e+00  4.19494452e-17 -5.63529833e-16
  1.91718703e-15  5.61321213e-16  3.29146935e-16 -6.12905438e-16
  3.77552668e-16  6.29779392e-16 -6.18967046e-17  4.21240072e-16
 -5.96356126e-16 -1.77759082e-16  8.94151189e-16 -1.52313758e-15
  9.92656144e-16  1.00653428e-15  3.77175341e-16  3.59161277e-17
 -1.19280573e-15  1.58463752e-15 -1.34363464e-15  2.92768557e-16
  2.12484766e-15 -4.05366473e-16  4.34400473e-16 -6.19114946e-16
 -8.89942263e-16 -7.48703684e-16  4.26931311e-16  6.18350760e-16
  5.15959689e-16 -8.86961508e-16  1.44087125e-16 -5.73333338e-16
 -1.713